<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250" /> 

# Cours TAL - Laboratoire 2
# Mise en œuvre et évaluation de *POS taggers* pour le français

**Objectif**

Appliquer des étiqueteurs morphosyntaxiques (POS taggers) disponibles dans NLTK et dans les outils Stanford NLP à des textes français, puis quantifier leurs performances.

**Instructions initiales**

* Télécharger l'archive `UD_French-GSD-withBlankLines.zip` fournie sur Cyberlearn.
* Placer les trois fichiers qu'elle contient dans le même dossier que le notebook.
* Ce sont des textes en français annotés avec les POS tags, provenant du projet ([Universal Dependencies](https://github.com/UniversalDependencies/UD_French-GSD)), et légèrement modifiés.
  - le fichier `fr-ud-train.conllu3` est destiné à l'entraînement
  - le fichier `fr-ud-dev.conllu3` est destiné aux tests préliminaires et aux réglages des paramètres
  - le fichier `fr-ud-test.conllu3` est destiné à l'évaluation finale.

**Questions préliminaires**

* En inspectant les fichiers, veuillez indiquer le numéro de la colonne où se trouvent les mots, et celui de la colonne où se trouvent leur étiquettes morpho-syntaxiques (*POS tags*).
* Veuillez chercher sur le Web la liste des *POS tags* du projet Universal Dependencies, avec leurs définitions, et indiquer l'URL ci-dessous.


**Réponses**
* 2 et 4 les premières sont (ID | FORM | LEMMA | UPOS | ...)
* https://universaldependencies.org/guidelines.html

**Question**
* Veuillez déterminer et afficher le nombre de tokens de chacun des trois fichiers.

In [15]:
def count_nb_tokens_in_conllu3_file(file_path):
    with open(file_path, 'r') as f:
        text = f.read()
        lines = text.split('\n')
        # Remove empty lines
        lines = [line for line in lines if line]
        nb_tokens = len(lines)
        return nb_tokens

print('Nombre de tokens dans le fichier d\'entrainement : ', count_nb_tokens_in_conllu3_file('fr-ud-train.conllu3'))
print('Nombre de tokens dans le fichier de test : ', count_nb_tokens_in_conllu3_file('fr-ud-test.conllu3'))
print('Nombre de tokens dans le fichier de validation : ', count_nb_tokens_in_conllu3_file('fr-ud-dev.conllu3'))


Nombre de tokens dans le fichier d'entrainement 366371
Nombre de tokens dans le fichier de test 10298
Nombre de tokens dans le fichier de validation 36830


## Partie 1 : Évaluer le Stanford POS tagger avec les modèles fournis pour le français

L'Université de Stanford fournit un étiqueteur morpho-syntaxique (POS tagger) qui utilise l'apprentissage automatique (https://nlp.stanford.edu/software/tagger.html) appelé Maxent Tagger.  Le tagger et ses modèles multilingues peuvent être téléchargés à l'URL ci-dessus (archive ZIP suivant le lien *Download > full Stanford Tagger version 3.9.2*, 130 MB environ).  

Pour simplifier, on vous propose de télécharger séparément le programme Java [stanford-postagger.jar](https://drive.switch.ch/index.php/s/hMY6yO7lmoQJuS3) et le modèle français [french-ud.tagger](https://drive.switch.ch/index.php/s/4HSqKRTTTkCgPfB) fournis par l'enseignant (mot de passe = reference).  Enregistrez ces deux fichiers dans le même dossier que ce notebook.

Le Maxent Tagger est en Java, et peut être exécuté depuis ce notebook avec un appel Java en ligne de commande.  Pour exécuter une commande système depuis le notebook, ajouter '!' devant (par exemple `! dir` ou `! ls`).  Utilisez la [documentation du Maxent Tagger](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/tagger/maxent/MaxentTagger.html), et plus précisément la section *Tagging and Testing from the command line*, pour comprendre comment l'invoquer.  Java doit être installé sur votre système, et si nécessaire, exécuter :
```python
import os
java_path = 'C:/Program Files (x86)/Java/jdk1.8.0_20/bin/java.exe'  # votre chemin de java.exe
os.environ['JAVA_HOME'] = java_path   # attention aux slash (pas backslash sous Windows)
```
*Note* : il est également possible d'appeler ce tagger avec des commandes NLTK grâce au module [nltk.tag.stanford](https://www.nltk.org/_modules/nltk/tag/stanford.html) mais la gestion des *paths* entre Java, les classes et les modèles peut être compliquée.

**Question**

Appliquez le Maxent Tagger pour étiqueter le fichier `fr-ud-dev.conllu3` et demandez à Maxent Tagger de mesurer la qualité par comparaison à une l'annotation de référence fournie dans le fichier. Quels sont les scores obtenus ?  Quel est le nombre le plus important?  Indiquez ces réponses en commentaires du code.

In [11]:
!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger -model french-ud.tagger -testFile format=TSV,wordColumn=1,tagColumn=3,fr-ud-dev.conllu3 -verboseResults false

Loading default properties from tagger french-ud.tagger
Loading POS tagger from french-ud.tagger ... done [0.3 sec].
Tagged 36830 words at 19132,47 words per second.
Model french-ud.tagger has xSize=304855, ySize=18, and numFeatures=104853.
Results on 1478 sentences and 36830 words, of which 3049 were unknown.
Total sentences right: 144 (9,742896%); wrong: 1334 (90,257104%).
Total tags right: 32360 (87,863155%); wrong: 4470 (12,136845%).
Unknown words right: 2232 (73,204329%); wrong: 817 (26,795671%).


---
On obtient les résultats suivants :
```
Précision sur l'ensemble des phrases : 9.7%
Précision sur l'ensemble des tags : 87.8%
Précision sur l'ensemble des mots inconus : 73.2%
```
Quel est le nombre le plus important ?
- La précision sur l'ensemble des tags est le plus important car ici on cherche à étiqueter les mots avec le bon tag.
---

De même, appliquez le Maxent Tagger pour étiqueter le fichier `fr-ud-test.conllu3` et indiquez la précision du tagger en commentaires du code (#).

In [12]:
!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger -model french-ud.tagger -testFile format=TSV,wordColumn=1,tagColumn=3,fr-ud-test.conllu3 -verboseResults false

# Précision sur l'ensemble des phrases : 12.9%
# Précision sur l'ensemble des tags : 87.0%
# Précision sur l'ensemble des mots inconnus : 69,8%

Loading default properties from tagger french-ud.tagger
Loading POS tagger from french-ud.tagger ... done [0.3 sec].
Tagged 10298 words at 15188,79 words per second.
Model french-ud.tagger has xSize=304855, ySize=18, and numFeatures=104853.
Results on 416 sentences and 10298 words, of which 697 were unknown.
Total sentences right: 54 (12,980769%); wrong: 362 (87,019231%).
Total tags right: 8960 (87,007186%); wrong: 1338 (12,992814%).
Unknown words right: 487 (69,870875%); wrong: 210 (30,129125%).


```
Précision sur l'ensemble des phrases : 12.9%
Précision sur l'ensemble des tags : 87.0%
Précision sur l'ensemble des mots inconnus : 69,8%
```
---
**Question subsidiare** : combien de phrases et de mots le tagger trouve-t-il dans les fichiers `fr-ud-dev.conllu3` et `fr-ud-test.conllu3` ?  Comparez avec votre propre estimation du nombre de mots.

**Réponse** :  
- `fr-ud-dev.conllu3` : 1'478 phrases et 36'830 mots. On avait compté 36'830 tokens.
- `fr-ud-test.conllu3` : 416 phrases et 10'298 mots. On avait compté 10'298 tokens.


## Partie 2 : Entraîner le Stanford POS tagger pour obtenir de nouveaux modèles

Le but de cette partie est d'entraîner le Maxent Tagger sur les données UD en français (`fr-ud-train.conllu3`), puis de comparer le modèle obtenu avec les modèles fournis par Stanford pour le français, testés dans la partie 1A.  

Suivre la [documentation de Maxent Tagger](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/tagger/maxent/MaxentTagger.html) pour l'entraîner sur le fichier `fr-ud-train.conllu3` et le tester sur `fr-ud-test.conllu3`.  Regardez la section *Training from the command line*. 

La configuration du système pour effectuer l'entraînement est donnée dans un fichier texte, qui peut être produit en suivant la documentation (option `-genprops` pour obtenir un template qui sera édité), soit en s'inspirant du fichier [french-ud.tagger.props](https://drive.switch.ch/index.php/s/gHlam9S74HG2Q4X) accompagnant le modèle `french-ud.tagger` que vous avez utilisé ci-dessus.  Pensez à donner un nouveau nom à votre fichier modèle.

**Questions**

* Créez un fichier `myFrench-ud.tagger.props` qui aboutit à un bon entraînement.  Vous pourrez expérimenter plusieurs fois et proposer le meilleur fichier.  Citez dans le notebook les paramètres sur lesquels vous avez agi.

* Lancez l'entraînement sur le fichier `fr-ud-train.conllu3` (s'il ne tient pas en mémoire, utilisez seulement `fr-ud-dev.conllu3`). Pendant l’entraînement (> 10 minutes, 500 itérations), regardez la suite du travail.

* Évaluez votre modèle comme ci-dessus (sur `dev` et sur `test`).  Quel modèle est meilleur, le vôtre ou celui fourni par Stanford ?  Formulez une hypothèse expliquant ce résultat. 

In [1]:
!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger -props myFrench-ud.tagger.props -mx1g

## tagger training invoked at Fri Mar 24 09:24:48 CET 2023 with arguments:
                   model = myFrench-ud.tagger
                    arch = left3words,naacl2003unknowns,unicodeshapes(-1,1)
            wordFunction = 
               trainFile = format=TSV,wordColumn=1,tagColumn=3,fr-ud-train.conllu3
         closedClassTags = 
 closedClassTagThreshold = 40
 curWordMinFeatureThresh = 2
                   debug = false
             debugPrefix = 
            tagSeparator = _
                encoding = utf-8
              iterations = 10
                    lang = french
    learnClosedClassTags = false
        minFeatureThresh = 2
           openClassTags = 
rareWordMinFeatureThresh = 10
          rareWordThresh = 5
                  search = qn
                    sgml = false
            sigmaSquared = 0.0
                   regL1 = 0.75
               tagInside = 
                tokenize = true
        tokenizerFactory = 
        tokenizerOptions = asciiQuotes
                

In [2]:
!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger -model myFrench-ud.tagger -testFile format=TSV,wordColumn=1,tagColumn=3,fr-ud-dev.conllu3 -verboseResults false

# Précision sur l'ensemble des phrases : 13.0%

Loading default properties from tagger myFrench-ud.tagger
## tagger testing invoked at Fri Mar 24 10:24:13 CET 2023 with arguments:
                   model = myFrench-ud.tagger
                    arch = left3words,naacl2003unknowns,unicodeshapes(-1,1)
            wordFunction = 
                testFile = format=TSV,wordColumn=1,tagColumn=3,fr-ud-dev.conllu3
         closedClassTags = 
 closedClassTagThreshold = 40
 curWordMinFeatureThresh = 2
                   debug = false
             debugPrefix = 
            tagSeparator = _
                encoding = utf-8
              iterations = 10
                    lang = french
    learnClosedClassTags = false
        minFeatureThresh = 2
           openClassTags = 
rareWordMinFeatureThresh = 10
          rareWordThresh = 5
                  search = qn
                    sgml = false
            sigmaSquared = 0.0
                   regL1 = 0.75
               tagInside = 
                tokenize = true
        tokenizerFactory = 


---
On obtient les résultats suivants :
```
Précision sur l'ensemble des phrases : 43.23%
Précision sur l'ensemble des tags : 95.7%
Précision sur l'ensemble des mots inconus : 83.1%
```
---

In [3]:
!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger -model myFrench-ud.tagger -testFile format=TSV,wordColumn=1,tagColumn=3,fr-ud-test.conllu3 -verboseResults false

Loading default properties from tagger myFrench-ud.tagger
## tagger testing invoked at Fri Mar 24 10:24:37 CET 2023 with arguments:
                   model = myFrench-ud.tagger
                    arch = left3words,naacl2003unknowns,unicodeshapes(-1,1)
            wordFunction = 
                testFile = format=TSV,wordColumn=1,tagColumn=3,fr-ud-test.conllu3
         closedClassTags = 
 closedClassTagThreshold = 40
 curWordMinFeatureThresh = 2
                   debug = false
             debugPrefix = 
            tagSeparator = _
                encoding = utf-8
              iterations = 10
                    lang = french
    learnClosedClassTags = false
        minFeatureThresh = 2
           openClassTags = 
rareWordMinFeatureThresh = 10
          rareWordThresh = 5
                  search = qn
                    sgml = false
            sigmaSquared = 0.0
                   regL1 = 0.75
               tagInside = 
                tokenize = true
        tokenizerFactory = 

---
On obtient les résultats suivants :
```
Précision sur l'ensemble des phrases : 40.3%
Précision sur l'ensemble des tags : 95%
Précision sur l'ensemble des mots inconus : 81.3%
```
---

## Partie 3 : entraîner un POS tagger pour le français dans NLTK

Le but de cette partie est d'utiliser le POS tagger *Averaged Perceptron* de NLTK, en l'entraînant pour le français sur les mêmes données que ci-dessus.  

Notez que pour l'anglais, des taggers pré-entraînés sont disponibles dans NLTK, comme expliqué au [Chapitre 5.1 du livre NLTK](http://www.nltk.org/book/ch05.html) : on peut écrire `nltk.pos_tag(sentence)` où *sentence* est une phrase tokenisée. L'étiquetage morpho-syntaxique produira des paires ('mot', 'TAG').

**Première étape**

Importer les textes annotés `fr-ud-XXXX.conllu3` grâce à des objets `ConllCorpusReader`.  Consultez le mode d'emploi de cette classe directement dans [son code source](https://www.nltk.org/_modules/nltk/corpus/reader/conll.html#ConllCorpusReader), pour déterminer comment lire un fichier en créant un objet `ConllCorpusReader`.  Chargez les trois fichiers, dans trois objets appelés `train_corpus`, `dev_corpus` et `test_corpus`.

In [96]:
from nltk.corpus.reader.conll import ConllCorpusReader
root = '.'
train_corpus = ConllCorpusReader(root, 'fr-ud-train.conllu3', columntypes=('ignore', 'words' , 'ignore', 'pos', 'ignore', 'ignore', 'ignore'), separator="\t")
test_corpus = ConllCorpusReader(root, 'fr-ud-test.conllu3', columntypes=('ignore', 'words' , 'ignore', 'pos', 'ignore', 'ignore', 'ignore'), separator="\t")
dev_corpus = ConllCorpusReader(root, 'fr-ud-dev.conllu3', columntypes=('ignore', 'words' , 'ignore', 'pos', 'ignore', 'ignore', 'ignore'), separator="\t")

Affichez le nombre de phrases et le nombre de mots de chaque corpus chargé. Cesc chiffres sont-ils identiques à ceux obtenus pour `dev` et pour `test` à la fin de la Partie 1 ?  On peut obtenir les listes de mots étiquetés avec `tagged_words()` et les listes de phrases avec mots étiquetés avec `tagged_sents()`.

In [100]:
print("Train corpus")
print("Number of sentences: ", len(train_corpus.sents()))
print("Number of words: ", len(train_corpus.words()))


print("Test corpus")
print("Number of sentences: ", len(test_corpus.sents()))
print("Number of words: ", len(test_corpus.words()))


print("Dev corpus")
print("Number of sentences: ", len(dev_corpus.sents()))
print("Number of words: ", len(dev_corpus.words()))

Train corpus
Number of sentences:  14554
Number of words:  366371
Test corpus
Number of sentences:  416
Number of words:  10298
Dev corpus
Number of sentences:  1478
Number of words:  36830


Affichez la 17e phrase du corpus de développement (avec les étiquettes POS), et les mots 1001 à 1050 du corpus de test (aussi avec leurs POS tags).

In [108]:
print("17th sentence of the dev corpus : \n ", dev_corpus.tagged_sents()[16])
print("Words 1001 to 1050 of the test corpus : \n ", test_corpus.tagged_words()[1001:1050])

17th sentence of the dev corpus : 
  [('Comprenant', 'VERB'), ('six', 'NUM'), ('sommets', 'NOUN'), ('dont', 'PRON'), ('un', 'DET'), ('point', 'NOUN'), ('culminant', 'VERB'), ('à', 'ADP'), ('2 001', 'NUM'), ('mètres', 'NOUN'), ('et', 'CCONJ'), ('une', 'DET'), ('arrivée', 'NOUN'), ('en', 'ADP'), ('altitude', 'NOUN'), (',', 'PUNCT'), ("c'", 'PRON'), ('est', 'AUX'), ('une', 'DET'), ('étape', 'NOUN'), ('typique', 'ADJ'), ('de', 'ADP'), ('montagne', 'NOUN'), ('.', 'PUNCT')]
Words 1001 to 1050 of the test corpus : 
  [('la', 'DET'), ('raison', 'NOUN'), ('politique', 'ADJ'), ('...', 'PUNCT'), ('Mais', 'CCONJ'), ('la', 'DET'), ('réalité', 'NOUN'), ('est', 'VERB'), ('que', 'SCONJ'), ('la', 'DET'), ('Mauritanie', 'PROPN'), ("n'", 'ADV'), ('est', 'AUX'), ('pas', 'ADV'), ('le', 'DET'), ('Maroc', 'PROPN'), ('ou', 'CCONJ'), ("l'", 'DET'), ('Algérie', 'PROPN'), ('.', 'PUNCT'), ('En', 'ADP'), ('Arabie', 'PROPN'), (',', 'PUNCT'), ('on', 'PRON'), ('a', 'VERB'), ("l'", 'DET'), ('impression', 'NOUN'), ('qu

**Seconde étape**

Vous allez maintenant entraîner (sur le corpus `train`) le POS tagger appelé *Averaged Perceptron* fourni par NLTK mais [implémenté par Mathew Honnibal de Explosion.AI](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python).

Dans le [package de NLTK avec des taggers](http://www.nltk.org/api/nltk.tag.html), considérez le module `nltk.tag.perceptron`, pour lequel NLTK explique de façon précise l'entraînement (voir *train the model*) et le test.  Vous allez mettre en oeuvre ces étapes pour entraîner le tagger.  Notez que le modèle est enregistré dans un fichier qui doit finir par `.pickle`, et qui est écrasé à chaque entraînement si vous ne changez pas de nom.  Un modèle peut être également chargé dans un tagger.

In [ ]:
# import os # si nécessaire
# import nltk # si nécessaire
# nltk.download('averaged_perceptron_tagger') # si nécessaire
from nltk.tag.perceptron import PerceptronTagger

In [ ]:
ptagger = PerceptronTagger(load=False)

Entraînez ici le tagger sur les données d'entraînement, avec les meilleurs paramètres possibles.

In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.


Combien de temps prend l'entraînement ?  Quelle est la taille du fichier modèle résultant ?

In [ ]:
# Veuillez écrire vos réponses dans cette cellule (en commentaires).
# 
# 

Évaluez le tagger, d'abord sur les données `dev` puis sur les données `test`.

In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.


Veuillez remplir le tableau suivant avec la synthèse des résultats.

| Corpus | MaxEnt | MaxEnt   | Avg Perceptron | 
|--------|--------|----------|---------------|
| -      | fourni | entraîné | entraîné |
| dev    |   ..   |   ..     |  ..  |
| test   |   ..   |   ..     |  ..  |

Comment se comparent les deux POS taggers sur le français ?  Écrivez vos conclusions dans cette cellule.

## Fin du laboratoire 2  

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, sauvegarder le résultat, et le rendre via Cyberlearn.